In [9]:
from re import T
import torchvision.transforms as transforms  #图像预处理工具
import torch
import numpy as np
from PIL import Image
from click.core import batch`
from matplotlib import pyplot as plt
import os

from torch.distributions import transform_to
from torch.utils.data import DataLoader, Dataset
from torch.xpu import device


In [10]:
#自定义数据类型
class ImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.main_dir = image_dir
        self.transform = transform
        self.image_names=os.listdir(image_dir)  #获取目录下所有文件名
        self.image_filenames = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]

    def __len__(self):
        return len(self.image_names)

#传入图片id，获取数据元素
    def __getitem__(self, idx):
        #根据索引号，构建图片的完整路径
        img_loc= os.path.join(self.main_dir, self.image_filenames[idx])
        image = Image.open(img_loc).convert('RGB')

        #利用transform转换成tensor
        if self.transform:
            image = self.transform(image)
        #抛出异常
        else:
            raise ValueError("Transform is not defined")

        #向原始图像中增加随机噪声
        noisy_factor = 0.5
        noisy_image = image + noisy_factor * torch.randn(*image.shape)
        noisy_image = torch.clamp(noisy_image, 0., 1.)
#返回噪声图片和原始图片
        return noisy_image, image




In [12]:
from torchvision.transforms import Compose
import torchvision.transforms as transforms  #图像预处理工具
 #测试主流程
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((68,68)),
    transforms.ToTensor()
])
#写代码时没有注意到中括号

#创建数据集
full_dataset=ImageDataset(image_dir ='../common/dataset/', transform=transform)

print(len(full_dataset))


24853


C:\Users\Augest\.conda\envs\image_en\lib\site-packages\torch\cuda\__init__.py:182: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10\cuda\CUDAFunctions.cpp:119.)
  return torch._C._cuda_getDeviceCount() > 0


In [15]:
import torch
from torch.utils.data import random_split
#划分数据集
#train_dataset, test_dataset = random_split(full_dataset[0.75,0.25])
train_dataset, test_dataset = random_split(full_dataset, [0.75, 0.25])
print(len(train_dataset))
print(len(test_dataset))

18640
6213


In [17]:
#创建dataLoder
batch_size=64
train_loder=DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True

)

In [18]:
test_loder=DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,

)

In [19]:
for x,y in train_loder:
   print(f"x: {x}, x shape: {x.shape}, x device: {x.device}")
   print(f"y: {y}, y shape: {y.shape}, y device: {y.device}")
   break

x: tensor([[[[0.8728, 0.5362, 0.4854,  ..., 0.4222, 0.7303, 0.4180],
          [0.0000, 1.0000, 1.0000,  ..., 1.0000, 0.2406, 1.0000],
          [0.0000, 0.0579, 0.3738,  ..., 1.0000, 1.0000, 0.3679],
          ...,
          [1.0000, 0.9772, 0.5151,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 0.9005, 0.0566,  ..., 0.6798, 1.0000, 1.0000],
          [0.8518, 1.0000, 1.0000,  ..., 1.0000, 0.0000, 0.6523]],

         [[0.9943, 1.0000, 0.7451,  ..., 0.4971, 1.0000, 0.6688],
          [1.0000, 1.0000, 0.7888,  ..., 0.4505, 0.8463, 1.0000],
          [0.8656, 1.0000, 1.0000,  ..., 0.6568, 1.0000, 0.6707],
          ...,
          [0.4943, 1.0000, 1.0000,  ..., 0.6562, 1.0000, 0.6767],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 0.4873, 0.4861],
          [0.5902, 0.1520, 0.3719,  ..., 0.7311, 1.0000, 0.8893]],

         [[1.0000, 0.5365, 1.0000,  ..., 0.7669, 0.3591, 1.0000],
          [1.0000, 1.0000, 0.0000,  ..., 0.6984, 0.3907, 1.0000],
          [0.8125, 1.0000, 0.8098,  ...

In [ ]:
from torch._refs import nn


#定义神经网络结构类
class ConvDenoiser(nn.Module):
    def __init__(self):
        super(ConvDenoiser,self).__init__()
        #编码器
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride =1,padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, stride =1,padding=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, stride =1,padding=1)
        self.pool =nn.MaxPool( 2,2)
        #解码器
        self.t_conv1 = nn.ConvTranspose2d(8,8,kernel_size=3,stride=2,padding=1)
        self.t_conv2 = nn.ConvTranspose2d(8,16,kernel_size=3,stride=2,padding=1)
         self.t_conv3= nn.ConvTranspose2d(16,32,kernel_size=3,stride=2,padding=1)
         self.conv_out= nn.Conv2d(32,3,3,padding=1)

def forward(self, x):

#编码
        x = torch.relu(self.t_conv1(x))
        print('conv1 shape:',x.shape)
        x = self.pool(x)
        print('pool1 shape:',x.shape)

        x = torch.relu(self.t_conv2(x))
        print('conv2 shape:',x.shape)
        x = self.pool(x)
        print('pool2 shape:',x.shape)

        x = torch.relu(self.t_conv3(x))
        print('conv3 shape:',x.shape)
        x = self.pool(x)
        print('pool3 shape:',x.shape)

#解码
       x=torch.relu(.conv1(x))
       print('conv1 shape:',x.shape)

        x=torch.relu(.conv2(x))
       print('conv1 shape:',x.shape)

        x=torch.relu(.conv3(x))
       print('conv1 shape:',x.shape)

       x=torch.sigmoid(self.conv_out(x))
       print('conv_out  shape:',x.shape)

       return  x


model = ConvDenoiser()
print(model)

x = torch.forward(torch.rand(1,3,68,68))






In [ ]:
#指定损失函数
